# Section 13: Trees


### Possible answers:

1) When horizontal gene transfer happens. Very common in phylogenetic analisys of prokaryots. For this reason people studying prokaryot often talk about "phylogenetic forest" rather than about separate phylogenetic trees.

2) Because DNA is double stranded it replicates the way it replicates, which means that more than two copies of a gene physically cannot originate from one ancestral copy. The fact that cells also cannot divide into more than two descendant cells (which is a consequence of replication process itself) also explains why phylogenetic trees are binary for some organisms. However, on the level of populations and species things get more complicated. In principal, we can divide a population of one species into three groups, simultaneously isolate them from each other and make three descendant species originate from them. But our methods of phylogenetic reconstruction would still represent this situation as a binary tree, although one of the branches will be very short.

3) Let's say we came up with a tree ((lion, cat), (mouse, rat), (human, chimp)). But what next? We cannot confidently say where is the root of this tree without any additional information. 

Cheap and unreliable way to find a root is **midpoint rooting**. To do that we need to 1) find the longest branch of the tree and 2) set the root in the middle of this branch. Try to avoid using this method unless you really need a rooted tree (which is actually rare!) and you are hopeless to find another way. The most common way to find a root is joinig an **outgroup**. Outgroup is a species coming from outside of our tree. If we add the outgroup sequences to the alignment and repeat the phylogenetic analisys, the branch where the outgroup will be connected to our tree is the root. For even more reliable results it is better to use more than one outgroup.

<figure> <center> <img src = "rooting.png" alt = "Finding a root of a tree." width="600" /> </center><br/>


### Weighted parsimony

Let us go on with this toy example, but instead consider only 4 species: lion, cat, mouse, rat. There are three possible tree topologies for this case. Here they are, with specified branch lengths we will need below.

<figure> <center> <img src = "trees.png" alt = "Toy example." width="600" /> </center><br/>


In [1]:
import numpy as np


# each tree is a list on leaves (first four) and internal nodes (three of them).
# node structure ((node name), (left child name, distance to the left child), (right child name, distance to the right child))

Trees = [ [ ('Lion', (), ()), ('Cat', (), ()), ('Mouse', (), ()), ('Rat', (), ()),  ('4', ('Lion', 0.1), ('Cat', 0.1)),  ('5', ('Mouse', 0.1), ('Rat', 0.1)),  ('6', ('4', 0.5), ('5', 0.5))],
          [ ('Lion', (), ()), ('Cat', (), ()), ('Mouse', (), ()), ('Rat', (), ()),  ('4', ('Lion', 0.1), ('Mouse', 0.1)),  ('5', ('Cat', 0.1), ('Rat', 0.1)),  ('6', ('4', 0.5), ('5', 0.5))],
          [ ('Lion', (), ()), ('Cat', (), ()), ('Mouse', (), ()), ('Rat', (), ()),  ('4', ('Lion', 0.1), ('Rat', 0.1)),  ('5', ('Mouse', 0.1), ('Cat', 0.1)),  ('6', ('4', 0.5), ('5', 0.5))] ]


Now let us imagine that in this world all the information about an organism is encoded with one symbol, in our case: 🦁, 🐈, 🐭, and 🐀. This is a weird world, but it will allow us to implement algorithms for only one alignment position, and in the homework you will just need to add iteration over alignment columns.

We will digitalize this alphabet just like we did with nucleotides:

|digitalized|native|
|-|----|
|0| 🦁 |
|1| 🐈 |
|2| 🐭 |
|3| 🐀 |

Therefore, here is our MSA:

In [2]:
msa = np.array([0,1,2,3])

In the weighted parsimony we will use matrix $\sigma$ to assign the costs. At each node $y$, we keep an array $S_{y}(a)$ for each residue $a$, which is the minimum weighted cost of the subtree rooted at $y$ if the ancestral residue at $y$ is an $a$.  We look at each possible residue $b$ in the left child $S_{lc}(b)$ and choose the minimum of that plus the cost $\sigma(b \mid a)$ for the $a \rightarrow b$ substitution; then we add the same calculation for the right child (from the lecture). As you run this algorithm, you can print out array $S$ to understand, what is going on

In [3]:
def weighted_parsimony(T, msa):
    
    T_dic = {e[0]: i for i, e in enumerate(T)}
    nnodes = len(T)
    ntaxa  = (nnodes + 1) // 2
    
    sigma = np.array(
      [[0., 0.1, 0.8, 0.8],
       [0.1, 0., 0.8, 0.8],
       [0.8, 0.8, 0., 0.1],
       [0.8, 0.8, 0.1, 0.]])

    S = np.zeros( (nnodes, 4) ) # initialization
    for y in range(ntaxa):
        for a in range(4):
            if msa[y] != a:
                S[y,a] = np.inf
#     print(S)
    for y in range(ntaxa, nnodes): # iterating over 3 internal nodes
        lc = T[y][1][0] # left child
        lc_branchlen = T[y][1][1]
        rc = T[y][2][0] # right child
        rc_branchlen = T[y][2][1]
        for b in range(4):
            S[y,b] = np.min(S[T_dic[lc]]) + sigma[np.argmin(S[T_dic[lc]]), b] + np.min(S[T_dic[rc]]) + sigma[np.argmin(S[T_dic[rc]]), b]
    cost = np.min(S[(2*ntaxa-2)])

#     print(S)
    return np.round(cost, 4)


Now we can compare the cost for all the trees:

In [4]:
print('{:>6} {:>10} {:>10}'.format('Tree 0', 'Tree 1', 'Tree 2'))
print('{:>6} {:>10} {:>10}'.format(weighted_parsimony(Trees[0], msa), weighted_parsimony(Trees[1], msa), weighted_parsimony(Trees[2], msa)))


Tree 0     Tree 1     Tree 2
   1.0        1.7        1.7


Tree 0 wins, common sense checks out!

### Maximum likelihood

For maximum likelihood implementation we need a substitution model. Here I am using an custom model (based on Kimura model) for the strange world in our example.

In [5]:
def subs_model(branchlen):
    
    "Substitution model P(b|a,t) given branch len in subst/site."
    
    alpha = 5
    beta = 1
    
    rt = 1/4 + 1/4 * np.exp(-4 * beta * branchlen) + 1/2 * np.exp(-2 * (alpha+beta) * branchlen)
    st = 1/4 - 1/4 * np.exp(-4 * beta * branchlen)
    ut = 1/4 + 1/4 * np.exp(-4 * beta * branchlen) - 1/2 * np.exp(-2 * (alpha+beta) * branchlen)
    P = np.array([[ rt, ut, st, st ],
                  [ ut, rt, st, st ],
                  [ st, st, rt, ut ],
                  [ st, st, ut, rt ]])
    return P


We keep an array $L_y(a)$ which is the probability of the subtree rooted at node $y$, with residue $a$ at that node.  We calculate that probability iteratively, by looking at connecting ancestor $a$ to all the possible identities for residues $b$ at the left child node, multiplying in a branch probability $P(b \mid a, t)$ with the previously calculated probability $L_w(b)$, and doing the same for the right child (from the lecture). Print out *Prob* and *L* arrays to understand what happens on each iteration.

In [6]:
# Felsenstein pruning algorithm

def maximum_likelihood(T, msa):
    
    T_dic = {e[0]: i for i, e in enumerate(T)}
    nnodes = len(T)
    ntaxa  = (nnodes + 1) // 2

    # precompute P(b|a,t) for each child branch
    Prob = []
    for y in range(ntaxa):         
        Prob.append(None)
    for y in range(ntaxa, nnodes):
        lc_branchlen = T[y][1][1]
        rc_branchlen = T[y][2][1]
        Prob.append( (subs_model(lc_branchlen), subs_model(rc_branchlen)) )
#     print(Prob)

    L = np.zeros( (nnodes, 4) ) # initialization  
    for y in range(ntaxa):  
        L[y,msa[y]] = 1.0
#     print(L)

    for y in range(ntaxa, nnodes): # iterating over 3 internal nodes
        
        lc = T[y][1][0] # left child
        rc = T[y][2][0] # right child
        
        lP = Prob[y][0]
        rP = Prob[y][1]
        
        for a in range(4):
            lprob,rprob = 0.0,0.0
            for b in range(4):
                lprob += L[T_dic[lc],b] * lP[a,b]
                rprob += L[T_dic[rc],b] * rP[a,b]
            L[y,a] = lprob * rprob
#     print(L)
    totprob = 0.
    for a in range(4):
        totprob += L[nnodes-1,a] * 0.25   # that's a uniform prior at the root
    logL = np.log(totprob) # negative logL
    
    return np.round(logL, 4)


In [7]:
print('{:>6} {:>10} {:>10}'.format('Tree 0', 'Tree 1', 'Tree 2'))
print('{:>6} {:>10} {:>10}'.format(maximum_likelihood(Trees[0], msa), maximum_likelihood(Trees[1], msa), maximum_likelihood(Trees[2], msa)))


Tree 0     Tree 1     Tree 2
-5.0859    -6.7384    -6.7384


Negative log likelihood is higher for Tree 0, checks out again! No matter, how strange this toy world is, biological common sense is still present there (because I tweaked substitution costs and model, of course).

Note how weighted parsimony becomes similar to maximum likelihood in comparison to Fitch parsimony. In the homework you don't need to implement weighted parsimony, instead, you need to figure out, how to simplify it to Fitch parsimony. And don't forget to return from the emoji world to DNA world!